In [1]:
import os

import requests
from bs4 import BeautifulSoup
from lxml import etree, html

In [2]:
def get_years_urls(base_url: str):
    main_url = "https://www.mapa.gob.es"
    response = requests.get(base_url)
    html_content = response.content

    tree = html.fromstring(html_content)

    ul_element = tree.xpath(
        '//*[@id="main"]/div/div[2]/div[2]/div/div/div[4]/div/div/ul'
    )

    urls = []

    if ul_element:
        ul_element = ul_element[0]
        for li in ul_element.iterchildren():
            relative_url = li.xpath(".//a")[0].get("href")
            urls.append(main_url + relative_url)
    else:
        print("Element not found")

    return urls

In [3]:
def get_area_and_production_urls(yearbook_urls: list[str]) -> list[str]:
    result = []

    area_and_production_xpath = (
        "//a[contains(text(), 'SUPERFICIES Y PRODUCCIONES DE CULTIVOS')]"
    )

    for yearbook_url in yearbook_urls:

        try:
            response = requests.get(yearbook_url)
            html_content = response.content

            tree = html.fromstring(html_content)

            res = tree.xpath(area_and_production_xpath)

            if res:
                res = res[0]
                area_and_production_url = res.get("href")

                result.append(yearbook_url + "/" + area_and_production_url)
            else:
                print(f"Nothing found for {yearbook_url}")
        except:
            print(f"Couldn't fetch {yearbook_url}")

    return result

In [4]:
base_url = "https://www.mapa.gob.es/es/estadistica/temas/publicaciones/anuario-de-estadistica/default.aspx"

In [5]:
yearbook_urls = get_years_urls(base_url)

In [6]:
area_and_production_urls = get_area_and_production_urls(yearbook_urls)